In [32]:
import qt
import requests
from qt import dt, np, pd, os
from pathlib import Path

In [3]:
EODHD_API_KEY	= os.environ.get('EODHD_API_KEY')
MASSIVE_API_KEY	= os.environ.get('MASSIVE_API_KEY')

In [4]:
# Configuration
API_KEY = MASSIVE_API_KEY
DOWNLOAD_DIR = "./massive_options_data"
BASE_URL = "https://api.polygon.io/v1/flatfiles/options/minute_aggs_v1"

# Create directory if it doesn't exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

In [5]:
def download_flat_files(months=1):
	end_date = dt.datetime.now()
	start_date = end_date - dt.timedelta(days=months * 30)
	
	current_date = start_date
	while current_date <= end_date:
		# Format the filename based on Massive's naming convention
		date_str = current_date.strftime("%Y-%m-%d")
		file_name = f"{date_str}.csv.gz"
		local_path = os.path.join(DOWNLOAD_DIR, file_name)
		
		# Massive structure: .../year/month/date.csv.gz
		year = current_date.year
		month = f"{current_date.month:02d}"
		
		# Construction of the direct download URL
		download_url = f"{BASE_URL}/{year}/{month}/{file_name}?apiKey={API_KEY}"
		
		print(f"Checking {date_str}...", end=" ")
		
		# Check if file already exists locally
		if os.path.exists(local_path):
			print("Already downloaded.")
		else:
			response = requests.get(download_url, stream=True)
			
			if response.status_code == 200:
				print(f"Downloading...")
				with open(local_path, "wb") as f:
					for chunk in response.iter_content(chunk_size=1024*1024):
						f.write(chunk)
			elif response.status_code == 404:
				print("Market closed (Weekend/Holiday).")
			else:
				print(f"Failed (Status: {response.status_code})")
		
		current_date += dt.timedelta(days=1)

if __name__ == "__main__":
	download_flat_files(1)

Checking 2026-01-05... Market closed (Weekend/Holiday).
Checking 2026-01-06... Market closed (Weekend/Holiday).
Checking 2026-01-07... Market closed (Weekend/Holiday).
Checking 2026-01-08... Market closed (Weekend/Holiday).
Checking 2026-01-09... Market closed (Weekend/Holiday).
Checking 2026-01-10... Market closed (Weekend/Holiday).
Checking 2026-01-11... Market closed (Weekend/Holiday).
Checking 2026-01-12... Market closed (Weekend/Holiday).
Checking 2026-01-13... Market closed (Weekend/Holiday).
Checking 2026-01-14... Market closed (Weekend/Holiday).
Checking 2026-01-15... Market closed (Weekend/Holiday).
Checking 2026-01-16... Market closed (Weekend/Holiday).
Checking 2026-01-17... Market closed (Weekend/Holiday).
Checking 2026-01-18... Market closed (Weekend/Holiday).
Checking 2026-01-19... Market closed (Weekend/Holiday).
Checking 2026-01-20... Market closed (Weekend/Holiday).
Checking 2026-01-21... Market closed (Weekend/Holiday).
Checking 2026-01-22... 

KeyboardInterrupt: 

In [6]:
import boto3
from botocore.config import Config

In [8]:
# Initialize a session using your credentials
session = boto3.Session(
	aws_access_key_id='5b4b35f0-0358-4c2e-9354-b12da27e8f96',
	aws_secret_access_key=MASSIVE_API_KEY,
)

# Create a client with your session and specify the endpoint
s3 = session.client(
	's3',
	endpoint_url='https://files.massive.com',
	config=Config(signature_version='s3v4'),
)

# Specify the bucket name
bucket_name = 'flatfiles'

In [33]:
def download_flat_1d(query_date = dt.date(2026,1,30), session=None):
	# Specify the S3 object key name
	# object_key = 'flatfiles/us_options_opra/minute_aggs_v1/2026/01/2026-01-30.csv.gz'
	object_key = query_date.strftime('flatfiles/us_options_opra/minute_aggs_v1/%Y/%m/%Y-%m-%d.csv.gz')

	# Remove the bucket name (e.g. 'flatfiles/') prefix if present in object_key
	if object_key.startswith(bucket_name + '/'):
		object_key = object_key[len(bucket_name + '/'):]

	# Specify the local file name and path to save the downloaded file
	local_file_name = object_key.split('/')[-1]  # e.g., '2025-06-12.csv.gz'
	local_file_path = './massive_options_data/' + local_file_name

	# Print the file being downloaded
	print(f"Downloading file '{object_key}' from bucket '{bucket_name}' to {local_file_path}")

	# ignore if file already exists
	if Path(local_file_path).is_file():
		qt.log.info(f"file for date {query_date} already exists, skip")
		return

	# Download the file
	try:
		session.download_file(bucket_name, object_key, local_file_path)
	
	except Exception as e:
		qt.log.error(f"error occurred while downloading flat file for date = {query_date}, erorr = {e}")

In [37]:
start_date = dt.date(2025,10,1)
end_date = dt.date.today()
dates = pd.date_range(start_date, end_date).date

In [39]:
for d in dates:
	download_flat_1d(query_date=d, session=s3)

[JUSTY.LOG]	2026-02-04 19:58:49,719 - qt.common.help - INFO - file for date 2025-10-01 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:49,721 - qt.common.help - INFO - file for date 2025-10-02 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:49,722 - qt.common.help - INFO - file for date 2025-10-03 already exists, skip


[JUSTY.LOG]	2026-02-04 19:58:52,287 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2025-10-04, erorr = An error occurred (404) when calling the HeadObject operation: Not Found


[JUSTY.LOG]	2026-02-04 19:58:52,506 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2025-10-05, erorr = An error occurred (404) when calling the HeadObject operation: Not Found
[JUSTY.LOG]	2026-02-04 19:58:52,507 - qt.common.help - INFO - file for date 2025-10-06 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:52,508 - qt.common.help - INFO - file for date 2025-10-07 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:52,508 - qt.common.help - INFO - file for date 2025-10-08 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:52,510 - qt.common.help - INFO - file for date 2025-10-09 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:52,510 - qt.common.help - INFO - file for date 2025-10-10 already exists, skip


[JUSTY.LOG]	2026-02-04 19:58:52,729 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2025-10-11, erorr = An error occurred (404) when calling the HeadObject operation: Not Found


[JUSTY.LOG]	2026-02-04 19:58:52,950 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2025-10-12, erorr = An error occurred (404) when calling the HeadObject operation: Not Found
[JUSTY.LOG]	2026-02-04 19:58:52,951 - qt.common.help - INFO - file for date 2025-10-13 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:52,952 - qt.common.help - INFO - file for date 2025-10-14 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:52,953 - qt.common.help - INFO - file for date 2025-10-15 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:52,954 - qt.common.help - INFO - file for date 2025-10-16 already exists, skip
[JUSTY.LOG]	2026-02-04 19:58:52,954 - qt.common.help - INFO - file for date 2025-10-17 already exists, skip


...

[JUSTY.LOG]	2026-02-04 19:59:00,120 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2026-01-19, erorr = An error occurred (404) when calling the HeadObject operation: Not Found
[JUSTY.LOG]	2026-02-04 19:59:00,120 - qt.common.help - INFO - file for date 2026-01-20 already exists, skip
[JUSTY.LOG]	2026-02-04 19:59:00,120 - qt.common.help - INFO - file for date 2026-01-21 already exists, skip
[JUSTY.LOG]	2026-02-04 19:59:00,121 - qt.common.help - INFO - file for date 2026-01-22 already exists, skip
[JUSTY.LOG]	2026-02-04 19:59:00,121 - qt.common.help - INFO - file for date 2026-01-23 already exists, skip


[JUSTY.LOG]	2026-02-04 19:59:00,335 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2026-01-24, erorr = An error occurred (404) when calling the HeadObject operation: Not Found


[JUSTY.LOG]	2026-02-04 19:59:00,541 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2026-01-25, erorr = An error occurred (404) when calling the HeadObject operation: Not Found
[JUSTY.LOG]	2026-02-04 19:59:00,542 - qt.common.help - INFO - file for date 2026-01-26 already exists, skip
[JUSTY.LOG]	2026-02-04 19:59:00,543 - qt.common.help - INFO - file for date 2026-01-27 already exists, skip
[JUSTY.LOG]	2026-02-04 19:59:00,543 - qt.common.help - INFO - file for date 2026-01-28 already exists, skip
[JUSTY.LOG]	2026-02-04 19:59:00,544 - qt.common.help - INFO - file for date 2026-01-29 already exists, skip
[JUSTY.LOG]	2026-02-04 19:59:00,544 - qt.common.help - INFO - file for date 2026-01-30 already exists, skip


[JUSTY.LOG]	2026-02-04 19:59:00,747 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2026-01-31, erorr = An error occurred (404) when calling the HeadObject operation: Not Found
[JUSTY.LOG]	2026-02-04 19:59:00,948 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2026-02-01, erorr = An error occurred (404) when calling the HeadObject operation: Not Found
[JUSTY.LOG]	2026-02-04 19:59:00,948 - qt.common.help - INFO - file for date 2026-02-02 already exists, skip


[JUSTY.LOG]	2026-02-04 19:59:00,949 - qt.common.help - INFO - file for date 2026-02-03 already exists, skip
[JUSTY.LOG]	2026-02-04 19:59:01,149 - qt.common.help - ERROR - error occurred while downloading flat file for date = 2026-02-04, erorr = An error occurred (403) when calling the HeadObject operation: Forbidden
